In [97]:
import pandas as pd
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from google.colab import drive


In [ ]:
#Function to calculate embeddings

In [8]:
#chunksize = 1000
#for chunk in pd.read_csv('large_file.csv', chunksize=chunksize):

df = pd.read_csv('amazon_review_small.txt', header=None)

#print(df.head());

In [9]:
# Function definintion to use USE encoder for embeddings

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed_using_USE(input):
  return model(input)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
#Sample of data in datafame
#df

In [23]:
#Split the data into test and train
df_test=df.sample(frac=0.2)
df_train=df.drop(df_test.index)

print("train shape", df_train.shape)
print("test shape", df_test.shape)

train shape (520000, 3)
test shape (130000, 3)


In [28]:
#write train and test to csv
drive.mount('drive')
df_test.to_csv("/content/drive/MyDrive/proj_data/df_test.csv")
df_train.to_csv("/content/drive/MyDrive/proj_data/df_train.csv")



Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
#read into dataframe from google drive
df_train = pd.read_csv('/content/drive/MyDrive/proj_data/df_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/proj_data/df_test.csv')

df_train

In [81]:
#Calculate embeddings
train_txt_1 = df_train.loc[:10000]["2"]
train_embed_1 = embed_using_USE(train_txt_1).numpy()


#train_with_embed_df=pd.concat([df_train,train_embed_1])

test_txt_1 = df_test.loc[:10000]["2"]
test_embed_1 = embed_using_USE(test_txt_1).numpy()



In [87]:
print("train embeddings shape" , train_embed_1.shape)

print("train embeddings size" , train_embed_1.size)
df_train_chunk=df_train.loc[:10000]
print("df train chunk size" , df_train_chunk.size)

#df_train_chunk

#add Embeddings to dataframe - training set
df_train_chunk[3]=train_embed_1.tolist()
#list(df_train_chunk.columns)
df_train_chunk.columns = ['row','ratings', 'short_note', 'long_note', 'embeddings']
df_train_chunk



train embeddings shape (10001, 512)
train embeddings size 5120512
df train chunk size 40004


<ipython-input-87-85b454fff290>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_chunk[3]=train_embed_1.tolist()


,row,ratings,short_note,long_note,embeddings
0,0,1,mens ultrasheer,"This model may be ok for sedentary types, but ...","[-0.04315425455570221, -0.045919716358184814, ..."
1,1,4,Surprisingly delightful,This is a fast read filled with unexpected hum...,"[-0.03691023588180542, 0.020594116300344467, 0..."
2,2,2,"Works, but not as advertised",I bought one of these chargers..the instructio...,"[0.004406762775033712, -0.038225993514060974, ..."
3,3,2,Oh dear,I was excited to find a book ostensibly about ...,"[-0.012829530984163284, -0.03760448098182678, ..."
4,4,2,Incorrect disc!,"I am a big JVC fan, but I do not like this mod...","[-0.037433382123708725, -0.06453002989292145, ..."
...,...,...,...,...,...
9996,12557,2,Overrated ... and wrong on some particulars.,The fact that this book is one of the most rec...,"[-0.02500542439520359, -0.01708340272307396, 0..."
9997,12559,3,Beware!,"Unless you are a Master or above, you will mos...","[-0.0421595573425293, -0.08135204017162323, 0...."
9998,12561,3,Does a good job telling you the ideas but with...,It is a strong point that this book is now in ...,"[-0.03954418748617172, -0.04465877264738083, 0..."
9999,12562,3,Flawed Masterpiece...,This is truly a great book for beginners and m...,"[-0.010767720639705658, -0.07414957880973816, ..."


In [88]:
#Same with test set
df_test_chunk=df_test.loc[:10000]
df_test_chunk[3]=test_embed_1.tolist()
df_train_chunk.columns = ['row','ratings', 'short_note', 'long_note', 'embeddings']

<ipython-input-88-e6231d4951e2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_chunk[3]=test_embed_1.tolist()


In [112]:
#Setup X and y
X=np.array(df_train_chunk['embeddings'].to_list())
y=np.array(df_train_chunk['ratings'].to_list())

#Split

#X_train, X_temp, y_train, y_temp = train_test_split(X,y, test_size=0.2,random_state = 42)

#X_test, X_val, y_test, y_val= train_test_split(X,y, test_size=0.2,random_state = 42)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state = 42)
print( X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(8000, 512) (2001, 512) (8000,) (2001,)


In [123]:
#Start fitting to models

#Logistic Regression
from sklearn.linear_model import LogisticRegression
regression_model = LogisticRegression(solver='lbfgs', max_iter=200)
regression_model.fit(X_train, y_train)



LogisticRegression(max_iter=200)

In [124]:
# Predict using the model and Evaluate the model
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
y_pred = regression_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

reg_mae = mean_absolute_error(y_test, y_pred)
reg_mse = mean_squared_error(y_test, y_pred)
reg_r_squared = r2_score(y_test, y_pred)
reg_rmse = np.sqrt(reg_mse)

# Print the evaluation metrics
print("Mean Absolute Error (MAE)", reg_mae)
print("Mean Squared Error (MSE)", reg_mse)
print("R2 Error (MAE)", reg_r_squared)
print("RMSE  Error (MAE)", reg_rmse)


Accuracy: 45.53%
Mean Absolute Error (MAE) 0.7901049475262368
Mean Squared Error (MSE) 1.4717641179410295
R2 Error (MAE) 0.2670920383289691
RMSE  Error (MAE) 1.2131628571387394


In [126]:
#Model using Nueral Network
import tensorflow as tf
import keras
from keras import layers
nn_model = keras.Sequential(
    [
        layers.Dense(30,activation="relu", name="layer1"),
        layers.Dense(10, activation="relu", name="layer2"),
        layers.Dense(1, name="layer3"),
    ]
)

In [127]:
nn_model.compile(optimizer='sgd', loss='mse')

In [129]:
nn_model.fit?

In [133]:
nn_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5482
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5372
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4954
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5284
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5138
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5187
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4841
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.4744
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5156
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.4727
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4525
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4707
Epoch 13/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4929
Epoch 14/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4571
Epoch 15/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1

In [134]:
y_pred = nn_model.predict(X_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [139]:
##Nueral network evaluation metrics
nn_mae = mean_absolute_error(y_test, y_pred)
nn_mse = mean_squared_error(y_test, y_pred)
nn_r_squared = r2_score(y_test, y_pred)
nn_rmse = np.sqrt(reg_mse)

#accuracy = nn_model.evaluate(X_test, y_test)
#print("Accuracy: ", accuracy)
# Print the evaluation metrics
print("Mean Absolute Error (MAE)", nn_mae)
print("Mean Squared Error (MSE)", nn_mse)
print("R2 Error (MAE)", nn_r_squared)
print("RMSE  Error (MAE)", nn_rmse)

Mean Absolute Error (MAE) 0.9621104955196619
Mean Squared Error (MSE) 1.5515115789739267
R2 Error (MAE) 0.22737945911768387
RMSE  Error (MAE) 1.2131628571387394
